Scraping Github Repos from multiple pages

In [31]:
import requests
import pandas as pd
import time

In [32]:
project_name = []
last_update = []

i = 1;
while i <= 96:
    
    cookies = {
        '_octo': 'GH1.1.1915587616.1681407000',
        '_device_id': '0a00f64228aef54abed6af711002f71a',
        'saved_user_sessions': '112911140%3AbF95nwVgtouvy3DObbW9mmpgR_j7rBjLqQuNFMZAnmzN4mVj',
        'user_session': 'bF95nwVgtouvy3DObbW9mmpgR_j7rBjLqQuNFMZAnmzN4mVj',
        '__Host-user_session_same_site': 'bF95nwVgtouvy3DObbW9mmpgR_j7rBjLqQuNFMZAnmzN4mVj',
        'logged_in': 'yes',
        'dotcom_user': 'dchen117',
        'has_recent_activity': '1',
        'color_mode': '%7B%22color_mode%22%3A%22auto%22%2C%22light_theme%22%3A%7B%22name%22%3A%22light%22%2C%22color_mode%22%3A%22light%22%7D%2C%22dark_theme%22%3A%7B%22name%22%3A%22dark%22%2C%22color_mode%22%3A%22dark%22%7D%7D',
        'preferred_color_mode': 'dark',
        'tz': 'America%2FNew_York',
        '_gh_sess': 'eqWHBV4rOfMFp5eJsxCdPaXbWR1zgh%2FfNq9eYJ%2BVOg7N4WOYIKKNws026%2F5IMxexxCjZhJsoF5TA%2BNJfNg4YUID9%2BWLAbmFjnm9N5DUimowDqYtFB8F6IXXA%2FQBnY57hSSqU370aR4zT9rwkRR8nb9NZmLH7W3wQ%2B0ivpYqYz3JYDyLEXTpSlnd%2BYThtfeyUoapDbxd%2B%2B9Ksw%2FU97n8JCU4Vep6d4BZAksLU%2FSyaF0Ij5FrrLzJAG0rYdhCeur%2BdgaXhYhkXz3Dz8Ohe%2Bnf8ONEU5f2VzKV5OzULH2ZpVnlDtAHCDRdmutAAEal%2BM8N5fRMMUD7q2Eb4ZfXMYjXaxL0ZQSKuUhb3FB6NtaBJVdCrM3W1CMMHfNg%2BHnIIyZ04rugdBQ%3D%3D--xbejwlxvS%2BRTknKH--%2BnahbZmVtHXZoKpqg3GTdw%3D%3D',
    }

    headers = {
        'authority': 'github.com',
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9',
        # 'cookie': '_octo=GH1.1.1915587616.1681407000; _device_id=0a00f64228aef54abed6af711002f71a; saved_user_sessions=112911140%3AbF95nwVgtouvy3DObbW9mmpgR_j7rBjLqQuNFMZAnmzN4mVj; user_session=bF95nwVgtouvy3DObbW9mmpgR_j7rBjLqQuNFMZAnmzN4mVj; __Host-user_session_same_site=bF95nwVgtouvy3DObbW9mmpgR_j7rBjLqQuNFMZAnmzN4mVj; logged_in=yes; dotcom_user=dchen117; has_recent_activity=1; color_mode=%7B%22color_mode%22%3A%22auto%22%2C%22light_theme%22%3A%7B%22name%22%3A%22light%22%2C%22color_mode%22%3A%22light%22%7D%2C%22dark_theme%22%3A%7B%22name%22%3A%22dark%22%2C%22color_mode%22%3A%22dark%22%7D%7D; preferred_color_mode=dark; tz=America%2FNew_York; _gh_sess=eqWHBV4rOfMFp5eJsxCdPaXbWR1zgh%2FfNq9eYJ%2BVOg7N4WOYIKKNws026%2F5IMxexxCjZhJsoF5TA%2BNJfNg4YUID9%2BWLAbmFjnm9N5DUimowDqYtFB8F6IXXA%2FQBnY57hSSqU370aR4zT9rwkRR8nb9NZmLH7W3wQ%2B0ivpYqYz3JYDyLEXTpSlnd%2BYThtfeyUoapDbxd%2B%2B9Ksw%2FU97n8JCU4Vep6d4BZAksLU%2FSyaF0Ij5FrrLzJAG0rYdhCeur%2BdgaXhYhkXz3Dz8Ohe%2Bnf8ONEU5f2VzKV5OzULH2ZpVnlDtAHCDRdmutAAEal%2BM8N5fRMMUD7q2Eb4ZfXMYjXaxL0ZQSKuUhb3FB6NtaBJVdCrM3W1CMMHfNg%2BHnIIyZ04rugdBQ%3D%3D--xbejwlxvS%2BRTknKH--%2BnahbZmVtHXZoKpqg3GTdw%3D%3D',
        'referer': 'https://github.com/search?q=stars%3A%3E23000&type=repositories&p=1',
        'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        'x-github-target': 'dotcom',
        'x-requested-with': 'XMLHttpRequest',
    }
    
    params = {
        'q': 'stars:>23000',
        'type': 'repositories',
        'p': str(i),
    }

    response = requests.get('https://github.com/search', params=params, cookies=cookies, headers=headers)
    
    try:
        # response variable
        response = requests.get('https://github.com/search', params=params, cookies=cookies, headers=headers) 
        response.raise_for_status()
        print(f"Page {i} - Status Code: {response.status_code}")
        
        # json object
        results_json = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed on page {i}: {e}")
        delay_seconds = 60  # default delay
        time.sleep(delay_seconds)
        continue
        
    except json.decoder.JSONDecodeError as e:
        print(f"JSONDecodeError on page {i}: {e}")
    
    project_list = results_json['payload']['results']
    
    for project in project_list:
        project_name.append(project['hl_name'])
        last_update.append(project['repo']['repository']['updated_at'])
    
    i += 1


Page 1 - Status Code: 200
Page 2 - Status Code: 200
Page 3 - Status Code: 200
Page 4 - Status Code: 200
Page 5 - Status Code: 200
Page 6 - Status Code: 200
Page 7 - Status Code: 200
Page 8 - Status Code: 200
Page 9 - Status Code: 200
Page 10 - Status Code: 200
Page 11 - Status Code: 200
Page 12 - Status Code: 200
Page 13 - Status Code: 200
Page 14 - Status Code: 200
Request failed on page 15: 429 Client Error: Too Many Requests for url: https://github.com/search?q=stars%3A%3E23000&type=repositories&p=15
Page 15 - Status Code: 200
Page 16 - Status Code: 200
Page 17 - Status Code: 200
Page 18 - Status Code: 200
Page 19 - Status Code: 200
Page 20 - Status Code: 200
Page 21 - Status Code: 200
Page 22 - Status Code: 200
Page 23 - Status Code: 200
Page 24 - Status Code: 200
Page 25 - Status Code: 200
Page 26 - Status Code: 200
Page 27 - Status Code: 200
Page 28 - Status Code: 200
Request failed on page 29: 429 Client Error: Too Many Requests for url: https://github.com/search?q=stars%3A%3E23

In [33]:
projects_df = pd.DataFrame({'Project':project_name, 'Last Update':last_update})

In [34]:
projects_df

,Project,Last Update
0,freeCodeCamp/freeCodeCamp,2024-02-14T04:15:40.091Z
1,EbookFoundation/free-programming-books,2024-02-10T10:28:51.115Z
2,sindresorhus/awesome,2024-02-13T23:06:10.906Z
3,public-apis/public-apis,2024-02-07T04:55:57.791Z
4,jwasham/coding-interview-university,2024-02-02T17:58:10.685Z
...,...,...
952,ogham/exa,2023-11-04T14:04:14.994Z
953,github/copilot-docs,2023-11-24T03:19:19.954Z
954,MostlyAdequate/mostly-adequate-guide,2023-12-08T10:40:21.060Z
955,VSCodium/vscodium,2024-02-11T16:35:58.034Z


In [35]:
projects_df.to_excel("projects_23001.xlsx", index=False)

try:
    with pd.ExcelWriter(
        "projects_9500AndUp.xlsx",
        mode="a",
        engine="openpyxl",
        if_sheet_exists="overlay",
    ) as writer:
         projects_df.to_excel(writer,sheet_name="Sheet1", startrow=writer.sheets["Sheet1"].max_row, index = False,header= False)
except FileNotFoundError:
    projects_df.to_excel("projects_9500AndUp.xlsx", index=False)